In [ ]:
import os
import json

from promptflow.core import AzureOpenAIModelConfiguration

# Initialize Azure OpenAI Connection
model_config = AzureOpenAIModelConfiguration(
        azure_deployment="gpt-4",
        api_version=os.environ["AZURE_OPENAI_API_VERSION"],
        azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"]
    )

In [ ]:
import pandas as pd

data_path = "../data/data.jsonl"

df = pd.read_json(data_path, lines=True)
df.head()

In [ ]:
from promptflow.client import load_flow

# load apology evaluatorfrom prompty
groundedness_eval = load_flow(source="prompty/groundedness.prompty", model={"configuration": model_config})
fluency_eval = load_flow(source="prompty/fluency.prompty", model={"configuration": model_config})
coherence_eval = load_flow(source="prompty/coherence.prompty", model={"configuration": model_config})
relevance_eval = load_flow(source="prompty/relevance.prompty", model={"configuration": model_config})

In [ ]:
import sys
sys.path.append('../contoso_chat')  # Replace '/path/to/contoso_chat' with the actual path to the 'contoso_chat' folder

from chat_request import get_response
from promptflow.evals.evaluators import RelevanceEvaluator

relevance_evaluator = RelevanceEvaluator(model_config)

In [ ]:
from promptflow.evals.evaluate import evaluate

result_eval = evaluate(
    data="../data/data.jsonl",
    target=get_response,
    evaluators={
        "relevance": relevance_eval,
        "fluency": fluency_eval,
        "coherence": coherence_eval,
        "groundedness": groundedness_eval,
    },
    # column mapping    return {"question": question, "answer": result, "context": context}
    evaluator_config={
        "default": {
            "question": "${data.question}",
            "answer": "${target.answer}",
            "context": "${target.context}",
        },
    },
)

In [ ]:
eval_result = pd.DataFrame(result_eval["rows"])

In [ ]:
eval_result.head()

In [ ]:
#save evaluation results to a JSONL file
eval_result.to_json('eval_result.jsonl', orient='records', lines=True)